In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/home/djawed/Documents/Portofolio/Spotify-reviews-sentiment-analysis/data/reviews.csv')
df.head()


,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


In [3]:
df = df.drop("Reply",axis=1)
df = df.dropna(subset=['Review']) 
df.head()

,Time_submitted,Review,Rating,Total_thumbsup
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1


In [4]:
df["Rating"] = np.where(df["Rating"] > 3, 1, 0)
df.head()

,Time_submitted,Review,Rating,Total_thumbsup
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",1,2
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,1,1
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",1,0
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,0,1
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,0,1


In [7]:
reviews = df["Review"]
y = df["Rating"].replace({1: 'positive', 0: 'negative'})
y

0        positive
1        positive
2        positive
3        negative
4        negative
           ...   
61589    negative
61590    negative
61591    negative
61592    negative
61593    positive
Name: Rating, Length: 61594, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re

In [86]:
X_train, X_test, y_train, y_test = train_test_split(reviews, y ,test_size = 0.2, random_state=42)

preprocessor = lambda text: re.sub(r'[^a-z ]', '', text.lower())

pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words="english",preprocessor=preprocessor)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='log_loss', random_state=42))
])


model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [87]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.2f}")


Model accuracy: 0.86


In [88]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

# Précision (ceux qui sont positifs et qui sont prédits comme positifs)
# Recall (Parmi ceux qui sont positifs, combien ont été prédits comme positifs)
# F1-Score (Moyenne entre précision et recall) 

              precision    recall  f1-score   support

    negative       0.84      0.90      0.87      6321
    positive       0.89      0.82      0.85      5998

    accuracy                           0.86     12319
   macro avg       0.87      0.86      0.86     12319
weighted avg       0.86      0.86      0.86     12319

